Для запуска следующих ноутбуков, если вы еще этого не сделали, необходимо указать ключ openai в файле .env как `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Далее мы загрузим индекс эмбеддингов в DataFrame Pandas. Индекс эмбеддингов хранится в JSON-файле с названием `embedding_index_3m.json`. Индекс эмбеддингов содержит эмбеддинги для каждой из транскрипций YouTube до конца октября 2023 года.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далее мы создадим функцию с названием `get_videos`, которая будет искать запрос в Индексе Встраиваний. Эта функция вернёт 5 видео, которые больше всего похожи на запрос. Работа функции происходит следующим образом:

1. Сначала создаётся копия Индекса Встраиваний.
2. Затем для запроса вычисляется Встраивание с помощью OpenAI Embedding API.
3. После этого в Индексе Встраиваний создаётся новый столбец с названием `similarity`. В этом столбце хранится косинусное сходство между Встраиванием запроса и Встраиванием каждого сегмента видео.
4. Далее Индекс Встраиваний фильтруется по столбцу `similarity`. Остаются только те видео, у которых косинусное сходство больше или равно 0.75.
5. В конце Индекс Встраиваний сортируется по столбцу `similarity`, и возвращаются 5 самых похожих видео.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Эта функция очень простая, она просто выводит результаты поискового запроса.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Сначала индекс встраиваний загружается в DataFrame библиотеки Pandas.
2. Затем пользователю предлагается ввести запрос.
3. Далее вызывается функция `get_videos`, чтобы найти запрос в индексе встраиваний.
4. После этого вызывается функция `display_results`, чтобы показать результаты пользователю.
5. Затем пользователю снова предлагается ввести запрос. Этот процесс продолжается, пока пользователь не введёт `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ru.png)

Вам будет предложено ввести запрос. Введите запрос и нажмите Enter. Приложение вернёт список видео, которые соответствуют вашему запросу. Также будет предоставлена ссылка на тот момент в видео, где находится ответ на ваш вопрос.

Вот несколько примеров запросов, которые можно попробовать:

- Что такое Azure Machine Learning?
- Как работают сверточные нейронные сети?
- Что такое нейронная сеть?
- Могу ли я использовать Jupyter Notebooks с Azure Machine Learning?
- Что такое ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
